In [ ]:
!pip -q install openai
!pip -q install tiktoken
!pip -q install langchain
!pip -q install deeplake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.1/789.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.8/476.8 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.8 MB/s eta 0:

In [ ]:

# imports
import ast  # for converting embeddings saved as strings back to arrays
import openai  # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
from scipy import spatial  # for calculating vector similarities for search


# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"


In [ ]:
import os
import getpass

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
os.environ['ACTIVELOOP_TOKEN'] = getpass.getpass('Activeloop Token:')

OpenAI API Key:··········
Activeloop Token:··········


In [ ]:
embeddings = OpenAIEmbeddings(disallowed_special=())

In [ ]:
!git clone https://github.com/srafay/Machine_Learning_A-Z # replace any repository of your choice 


Cloning into 'Machine_Learning_A-Z'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 243 (delta 1), reused 2 (delta 0), pack-reused 238
Receiving objects: 100% (243/243), 11.10 MiB | 27.12 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [ ]:
import os
from langchain.document_loaders import TextLoader

root_dir = '/content/Machine_Learning_A-Z'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
  n = 0
  for file in filenames:
    
        try: 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
            print 
        except Exception as e: 
            pass

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(docs)


In [ ]:
len(texts)

2539

In [ ]:
username = "sarthmirashi" # replace with your username from app.activeloop.ai
db = DeepLake(dataset_path=f"hub://{username}/MLAZ_RCTS", embedding_function=embeddings, public=True) #dataset would be publicly available
db.add_documents(texts)

In [ ]:
db = DeepLake(dataset_path="hub://sarthmirashi/MLAZ_RCTS", read_only=True, embedding_function=embeddings)
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10

def filter(x):
    # filter based on source code
    if 'com.google' in x['text'].data()['value']:
        return False
    
    # filter based on path e.g. extension
    metadata =  x['metadata'].data()['value']
    return 'scala' in metadata['source'] or 'py' in metadata['source']

### turn on below for custom filtering
# retriever.search_kwargs['filter'] = filter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name = 'gpt-3.5-turbo') # switch to 'gpt-4'

qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/sarthmirashi/MLAZ_RCTS



-

hub://sarthmirashi/MLAZ_RCTS loaded successfully.

Deep Lake Dataset in hub://sarthmirashi/MLAZ_RCTS already exists, loading from the storage
Dataset(path='hub://sarthmirashi/MLAZ_RCTS', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape       dtype  compression
  -------   -------    -------     -------  ------- 
 embedding  generic  (2539, 1536)  float32   None   
    ids      text     (2539, 1)      str     None   
 metadata    json     (2539, 1)      str     None   
   text      text     (2539, 1)      str     None   


In [ ]:

questions = [
    "do i standardise before spliting the data or after ? ",
] 
chat_history = []
result = qa({"question": questions[0], "chat_history": chat_history})

print(result['answer'])

It is recommended to perform feature scaling (standardization) after splitting the data into training and testing sets. This is to avoid data leakage from the test set into the training set, which can lead to overestimating the performance of the model.


In [ ]:

questions = [
    "how do i use the confusion matrix",
] 
chat_history = []
result = qa({"question": questions[0], "chat_history": chat_history})

print(result['answer'])

The confusion matrix is a way to evaluate the performance of a classification model. It shows the number of correct and incorrect predictions made by the model compared to the actual outcomes. To use the confusion matrix, you will need to have the predicted values and the true values. You can then use the `table()` function in R to create the matrix. The rows represent the true values, and the columns represent the predicted values. The diagonal represents the number of correct predictions, while the off-diagonal elements represent the incorrect predictions.


In [ ]:
questions = [
   "do i standardise before spliting the data or after ? ",
   "can you elaborate?",
   "what function can i use to do this",
   "can you give me the code that i can apply"
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: do i standardise before spliting the data or after ?  

**Answer**: It is generally recommended to standardize the data after splitting into training and test sets. This is because scaling should be applied only on the training data and then the same scaling parameters should be used on the test data. 



-> **Question**: can you elaborate? 

**Answer**: It is recommended to standardize the data after splitting into training and test sets to avoid information leakage. If the data is standardized before splitting, the mean and variance of the whole dataset is used to transform the test set which is wrong. This may lead to overfitting as the model may perform very well on the training set but poorly on the test set. 



-> **Question**: what function can i use to do this 

**Answer**: The function that should be used to standardize the data after splitting into training and test sets is "scale()". 



-> **Question**: can you give me the code that i can apply 

**Answer**: To standardize data after splitting into training and test sets, you should use the following code:

```
training_set[-dependent_variable_column] = scale(training_set[-dependent_variable_column])
test_set[-dependent_variable_column] = scale(test_set[-dependent_variable_column])
```

Replace `dependent_variable_column` with the column index or name of the dependent variable in your dataset. 

